Embedding Sypnosis

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
df_anime = pd.read_csv(r"G:\hoc\private\Anime\data\Recomended_Anime_data\raw\MyAnimeList-Database-master\data\anime_with_synopsis.csv")

In [ ]:
model = SentenceTransformer("paraphrase-mpnet-base-v2")

In [ ]:
embeddings = model.encode(df_anime["sypnopsis"].tolist(), show_progress_bar=True, convert_to_tensor=False)

Batches:   0%|          | 0/507 [00:00<?, ?it/s]

In [ ]:
np.save(r"G:\hoc\private\Anime\notebooks\Recomended_Anime_data/anime_embeddings.npy", embeddings)

In [3]:
%pip install openai

   ---------------------------------------- 0.0/928.2 kB ? eta -:--:--
   ---------------------------------------- 928.2/928.2 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import numpy as np
# import pandas as pd
# from openai import OpenAI

# # Load dataset
# df_anime = pd.read_csv(
#     r"G:\hoc\private\Anime\data\Recomended_Anime_data\raw\MyAnimeList-Database-master\data\anime_with_synopsis.csv"
# )

# # Initialize OpenAI client
# client = OpenAI(api_key="YOUR_OPENAI_API_KEY")

# # Extract the synopsis column (note: your column seems to be misspelled "sypnopsis")
# texts = df_anime["sypnopsis"].fillna("").tolist()

# # Generate embeddings in batches (to avoid hitting token/input limits)
# batch_size = 1000
# embeddings = []

# for i in range(0, len(texts), batch_size):
#     batch = texts[i:i+batch_size]
#     response = client.embeddings.create(
#         model="text-embedding-3-large",
#         input=batch
#     )
#     batch_embeddings = [item.embedding for item in response.data]
#     embeddings.extend(batch_embeddings)

# # Convert to numpy array and save
# embeddings = np.array(embeddings)
# np.save(r"G:\hoc\private\Anime\notebooks\Recomended_Anime_data\text-embedding-3-large.npy", embeddings)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: YOUR_OPE*******_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

[('Cowboy Bebop: Tengoku no Tobira', 0.67239034),
 ('Ginga Reppuu Baxingar', 0.594574),
 ('Xevious', 0.5573846),
 ('Macross F', 0.55287504),
 ('Star Fox Zero: The Battle Begins', 0.5510756)]

<p style="font-size:40px; color:White;">II. Calling embedding npz</p>

In [ ]:
# %pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.2 MB 2.8 MB/s eta 0:00:07
   -- ------------------------------------- 1.3/18.2 MB 2.7 MB/s eta 0:00:07
   ---- ----------------------------------- 1.8/18.2 MB 2.5 MB/s eta 0:00:07
   ----- ---------------------------------- 2.4/18.2 MB 2.4 MB/s eta 0:00:07
   ----- ---------------------------------- 2.6/18.2 MB 2.3 MB/s eta 0:00:07
   ------ --------------------------------- 3.1/18.2 MB 2.2 MB/s eta 0:00:07
   -------- ------------------------------- 3.7/18.2 MB 2.2 MB/s eta 0:00:07
   --------- ------------------------------ 4.2/18.2 MB 2.3 MB/s eta 0:00:07
   --------- ------------------------------ 4.5/18.2 MB 2.3 MB/s eta 0:00:07
   ---------- ----------------------------- 5.0/18.2 MB 2.2 MB/s eta 0:00:07
   ---------- ----------------------------- 5.0/18.2 MB 2.2 MB/s eta 0:00:07
   ----------

In [5]:
import faiss

def recommend_by_embedding(Name, anime_df, embeddings, top_n=5):
    idx = anime_df.index[anime_df["Name"] == Name][0]
    query_vec = embeddings[idx].reshape(1, -1)
    
    sim_scores = cosine_similarity(query_vec, embeddings)[0]
    top_idx = sim_scores.argsort()[-top_n-1:][::-1]  # sort high → low
    
    results = []
    for i in top_idx:
        if i != idx:  # skip the anime itself
            results.append((anime_df.iloc[i]["Name"], sim_scores[i]))
    return results


In [6]:
df_anime = pd.read_csv(r"G:\hoc\private\Anime\data\Recomended_Anime_data\raw\MyAnimeList-Database-master\data\anime_with_synopsis.csv")

In [7]:
embeddings = np.load("anime_embeddings.npy").astype("float32")
faiss.normalize_L2(embeddings)

# Build FAISS index
d = embeddings.shape[1]  # embedding dimension
index = faiss.IndexFlatIP(d)  # Inner Product works with normalized vectors
index.add(embeddings)

# Save FAISS index
faiss.write_index(index, "anime_index.faiss")



In [8]:
faiss.normalize_L2(embeddings)
embeddings = np.load("anime_embeddings.npy").astype("float32")

# Load FAISS index
index = faiss.read_index("anime_index.faiss")

def recommend_faiss(anime_idx, k=5):
    query = embeddings[anime_idx].reshape(1, -1).astype("float32")
    faiss.normalize_L2(query)  # normalize query too
    scores, indices = index.search(query, k+1)  # k+1 to skip itself
    return df_anime.iloc[indices[0][1:k+1]][["Name", "Score", "Genres"]]

# Example: recommend 5 similar anime to anime at index 10
print(recommend_faiss(10, k=5))

# cos_sim = cosine_similarity(embeddings)
# recommend_by_embedding("Cowboy Bebop", df_anime, embeddings, top_n=5)

                                             Name Score  \
1508                           Naruto: Shippuuden  8.16   
11346             Boruto: Naruto Next Generations  5.81   
5487    Naruto: Shippuuden Movie 5 - Blood Prison  7.46   
8831                     Boruto: Naruto the Movie   7.5   
6158   Naruto: Shippuuden Movie 6 - Road to Ninja  7.67   

                                                  Genres  
1508   Action, Adventure, Comedy, Super Power, Martia...  
11346  Action, Adventure, Super Power, Martial Arts, ...  
5487   Action, Adventure, Martial Arts, Super Power, ...  
8831   Action, Comedy, Martial Arts, Shounen, Super P...  
6158   Action, Adventure, Super Power, Martial Arts, ...  


In [18]:
Name_to_idx = {Name: i for i, Name in enumerate(df_anime["Name"])}
embeddings = np.load("anime_embeddings.npy").astype("float32")

# Load FAISS index
index = faiss.read_index("anime_index.faiss")

In [19]:
def recommend_by_name(title, k=5):
    if title not in Name_to_idx:
        raise ValueError(f"'{title}' not found in dataset")

    # Get index of this anime
    anime_idx = Name_to_idx[title]

    # Build query vector
    query = embeddings[anime_idx].reshape(1, -1).astype("float32")
    faiss.normalize_L2(query)

    # Search in FAISS
    scores, indices = index.search(query, k+1)  # k+1 to skip itself
    indices = indices[0][1:]   # drop itself
    scores = scores[0][1:]

    # Return DataFrame with similarity scores
    result = df_anime.iloc[indices][["Name", "Score", "Genres"]].copy()
    result["similarity"] = scores
    return result.reset_index(drop=True)

In [20]:
print(recommend_by_name("Initial D Fifth Stage", k=5))

                              Name    Score  \
0            Initial D First Stage     8.29   
1            Initial D Third Stage     7.89   
2  Arrow Emblem Grand Prix no Taka     6.47   
3           Initial D Second Stage     8.12   
4         Tobidase! Machine Hiryuu  Unknown   

                                         Genres  similarity  
0           Action, Cars, Drama, Seinen, Sports    0.607040  
1  Action, Cars, Sports, Drama, Romance, Seinen    0.597591  
2                           Cars, Sports, Drama    0.580465  
3           Action, Cars, Drama, Seinen, Sports    0.577739  
4                 Cars, Comedy, Parody, Shounen    0.556803  


In [11]:
# %pip install rapidfuzz

In [12]:
from rapidfuzz import process

def find_closest_title(query, titles, limit=1, score_cutoff=70):
    """
    Fuzzy match a user query to the closest title in the dataset.
    """
    matches = process.extract(query, titles, limit=limit, score_cutoff=score_cutoff)
    if not matches:
        return None
    # return best match
    return matches[0][0]   # (title, score, index) → take title


In [13]:
def recommend_by_name_fuzzy(query, k=5):
    # Find closest anime title in dataset
    best_match = find_closest_title(query, df_anime["Name"].tolist())
    if best_match is None:
        raise ValueError(f"No close match found for '{query}'")

    # Lookup index
    anime_idx = Name_to_idx[best_match]

    # Build query vector
    qvec = embeddings[anime_idx].reshape(1, -1).astype("float32")
    faiss.normalize_L2(qvec)

    # Search FAISS
    scores, indices = index.search(qvec, k+1)
    indices = indices[0][1:]
    scores = scores[0][1:]

    # Return results
    result = df_anime.iloc[indices][["Name", "Score", "Genres"]].copy()
    result["similarity"] = scores
    return best_match, result.reset_index(drop=True)

In [15]:
query = "Love Live"   # user typo or different wording
match, recs = recommend_by_name_fuzzy(query, k=5)

print("You searched for:", query)
print("Best match in dataset:", match)
print(recs)

You searched for: Love Live
Best match in dataset: Love Live! School Idol Project
                                                Name Score  \
0          Love Live! School Idol Project 2nd Season  7.81   
1                   Love Live! The School Idol Movie  7.95   
2  Love Live! Nijigasaki Gakuen School Idol Douko...  7.62   
3                              Love Live! Sunshine!!   7.4   
4                   Love Live! Sunshine!! 2nd Season  7.58   

                         Genres  similarity  
0  Music, School, Slice of Life    0.737099  
1  Music, School, Slice of Life    0.697409  
2  Music, Slice of Life, School    0.675268  
3  Music, Slice of Life, School    0.655351  
4  Music, School, Slice of Life    0.640358  
